# 使用 LangChain 构建一个 RAG 应用

## RAG 是什么

RAG 是一种将检索到的文档上下文与大语言模型（LLM）结合起来生成答案的技术。

整个过程主要分为以下几个步骤：

1. 加载文档：将原始数据(来源可能是在线网站、本地文件、各类平台等)加载到 LangChain 中。
1. 文档分割：将加载的文档分割成较小的块，以适应模型的上下文窗口，并更容易进行向量嵌入和检索。
1. 存储嵌入：将分割后的文档内容嵌入到向量空间，并存储到向量数据库中，以便后续检索。
1. 检索文档：通过查询向量数据库，检索与问题最相关的文档片段。
1. 生成回答：将检索到的文档片段与用户问题组合，生成并返回答案。

通过这些步骤，可以构建一个强大的问答系统，将复杂任务分解为更小的步骤并生成详细回答。

![rag](../images/rag.png)

In [1]:
!pip install langchain langchain_community langchain_chroma

^C


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ------------------------------- -------- 0.8/1.0 MB 4.2 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
     --------------------- ------------------ 1.3/2.4 MB 6.1 MB/s eta 0:00:01
     ---------------------------------------- 2.4/2.4 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ------------------------- -------------- 1.3/2.1 MB 6.1 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     -- ------------------------------------- 1.0/15.8 MB 5.6 MB/s eta 0:00:03
     ----- ---------------------------------- 2.1/15.8 MB 5.6 MB/s eta 0:00:03
     -------- --------------

## **RAG 开发指南**

**本指南将详细介绍如何使用 LangChain 框架构建一个基于检索增强生成 (RAG) 的应用。**

下面是基于 LangChain 实现的 RAG 的核心步骤与使用到的关键代码抽象（类型、方法、库等）:

1. **加载文档**: 使用 `WebBaseLoader` 类从指定来源加载内容，并生成 `Document` 对象（依赖 `bs4` 库）。
2. **文档分割**: 使用 `RecursiveCharacterTextSplitter` 类的 `split_documents()` 方法将长文档分割成较小的块。
3. **存储嵌入**: 使用 `Chroma` 类的 `from_documents()` 方法将分割后的文档内容嵌入向量空间，并存储在向量数据库中（使用 `OpenAIEmbeddings`），并可以通过检查存储的向量数量来确认存储成功。。
4. **检索文档**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法基于查询从向量数据库中检索最相关的文档片段。
5. **生成回答**: 使用 `ChatOpenAI` 类的 `invoke()` 方法，将检索到的文档片段与用户问题结合，生成回答（通过 `RunnablePassthrough` 和 `StrOutputParser`）。

我们使用的文档是Lilian Weng撰写的《LLM Powered Autonomous Agents》博客文章（https://lilianweng.github.io/posts/2023-06-23-agent/ ），最终构建好的 RAG 应用支持我们询问关于该文章内容的相关问题。

In [1]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

USER_AGENT environment variable not set, consider setting it to identify your requests.


### Step 1: 加载文档

- **描述**: 使用 `DocumentLoader` 从指定来源（如网页）加载内容，并将其转换为 `Document` 对象。
- **重要代码抽象**:
  - 类: `WebBaseLoader`
  - 方法: `load()`
  - 库: `bs4` (BeautifulSoup)
- **代码解释**:
  - **文档加载**: 使用 `WebBaseLoader` 从网页加载内容，并通过 `BeautifulSoup` 解析 HTML，提取重要的部分。
  - **检查加载数量**: 打印加载的文档数量，确保所有文档正确加载。
  - **验证文档内容**: 输出第一个文档的部分内容，确认加载的数据符合预期。

In [5]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("subject", "related_info"))
loader = WebBaseLoader(
    web_paths=("https://m.douban.com/book/subject/24531956/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [6]:
# 检查加载的文档内容长度
print(docs)  # 打印第一个文档内容的长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

[Document(metadata={'source': 'https://m.douban.com/book/subject/24531956/'}, page_content='\n\n内容简介\n      \xa0·\xa0·\xa0·\xa0·\xa0·\xa0·\n\n  \n\n\n\n\n《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使...(展开全部)\n\n\n\n\n\n《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使哈利波特系列7本浑然一体，必将成为每一个喜欢哈利·波特读者爱不释手的珍藏！     《哈利·波特与魔法石》     一岁的哈利·波

In [10]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:300])



内容简介
       · · · · · ·

  




《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。  


### Step 2: 文档分割

- **描述**: 使用文本分割器将加载的长文档分割成较小的块，以便嵌入和检索。
- **重要代码抽象**:
  - 类: `RecursiveCharacterTextSplitter`
  - 方法: `split_documents()`
- **代码解释**:
  - **文档分割**: 使用 `RecursiveCharacterTextSplitter` 按字符大小分割文档块，设置块大小和重叠字符数，确保文档块适合模型处理。
  - **检查块数量**: 打印分割后的文档块数量，确保分割操作正确执行。
  - **验证块大小**: 输出第一个块的字符数，确认分割块的大小是否符合预期。

In [11]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [12]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

16


In [13]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

421


In [14]:
print(all_splits[0].page_content)  # 打印第一个块的内容

内容简介
       · · · · · ·

  




《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使...(展开全部)


In [15]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'https://m.douban.com/book/subject/24531956/', 'start_index': 2}


### Step 3: 存储嵌入

- **描述**: 将分割后的文档内容嵌入到向量空间中，并存储到向量数据库，以便后续检索。
- **重要代码抽象**:
  - 类: `Chroma`
  - 方法: `from_documents()`
  - 类: `OpenAIEmbeddings`
- **代码解释**:
  - **存储嵌入**: 使用 `Chroma.from_documents()` 方法将所有分割的文档片段进行嵌入(`OpenAIEmbeddings`嵌入模型)，将文档片段嵌入向量空间，并存储在向量数据库中。

#### Chroma 基础使用

**下面是初始化 Chroma 数据库（仅实例化，未存储向量数据）的常见做法：**

**使用构造函数初始化**: 在本地持久化存储 Chroma 数据库.

```python
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)
```

**使用 Cleint 初始化**: 更方便地访问底层数据库/集合。

```python
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)
```


**我们直接使用 `Chroma.from_documents()` 方法 实例化+数据存储**:

该方法返回 Chroma 实例，数据类型为`langchain_chroma.vectorstores.Chroma`，详细 API 文档： https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html

In [17]:
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings()
)

In [18]:
# 查看 vectorstore 数据类型
type(vectorstore) 

langchain_chroma.vectorstores.Chroma

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [19]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [20]:
type(retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [23]:
retrieved_docs = retriever.invoke("what is the name of the book?")

In [24]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

6


In [25]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

内容简介
       · · · · · ·

  




《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使...(展开全部)


In [98]:
retrieved_docs = retriever.invoke("what is the author of the book?")
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

6
哈利·波特的创作者
       · · · · · ·

  










J·K·罗琳
作者







马爱农
译者







马爱新
译者







作者简介
       · · · · · ·

  




J. K. 罗琳（J. K. Rowling， 1965- ），英国女作家，自小喜欢写作，当过短时间的教师和秘书。二十四岁那年，她在前往伦敦的火车旅途中萌生了创作“哈利·波特”系列小说的念头。七年后，《哈利·波特与魔法石》问世，之后她陆续创作了《哈利·波特与密室》《哈 利·波特与阿兹卡班囚徒》《哈利·波特与火焰杯》《哈利-波特与凤凰社》《哈利·波特与“混血王子”》和《哈利·波特与死亡圣器》，完成了该系列的创作，在全球刮起一股股“哈利·波特”飓风，在世界范围内掀起规模宏大的“哈利·波特”阅读狂潮。另外，在此期间，她还因为慈善事业而先后完成了《神奇动物在哪里》和《神奇的魁地奇球》这两部与“哈利·波特”系列相关的图书。 2012年推出首部成人长篇小说《偶发空缺》。 J. K. 罗琳与她的丈夫及三个孩子生活在爱丁堡，一边尽享家庭生活的幸福，一边文学创作。



目录
       · · · · · ·

  

        《哈利·波特与魔法石》
        《哈利·波特与密室》
        《哈利·波特与阿兹卡班的囚徒》
        《哈利·波特与火焰杯》
        《哈利·波特与凤凰社》
        《哈利·波特与“混血王子”》
    · · · · · ·
    (更多)


        《哈利·波特与魔法石》
        《哈利·波特与密室》
        《哈利·波特与阿兹卡班的囚徒》
        《哈利·波特与火焰杯》
        《哈利·波特与凤凰社》
        《哈利·波特与“混血王子”》
        《哈利·波特与死亡圣器》
     · · · · · ·     (收起)














      原文摘录
        · · · · · · 
      ( 全部 )




            沉湎于虚幻的梦想，而忘记现实的生活，这是毫无益处的，千万记住。 (查看原文)


In [101]:
retrieved_docs = retriever.invoke("what is the publish year of this book?")
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

6
内容简介
       · · · · · ·

  




《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使...(展开全部)


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [26]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(model="gpt-4o-mini")

In [27]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("rlm/rag-prompt")

e:\code\github\DjangoPeng\openai-quickstart\.venv\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [28]:
# 打印模板
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [29]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [30]:
# 查看提示词
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [63]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
# 流式生成回答
for chunk in rag_chain.stream("What is the book abou?"):
    print(chunk, end="", flush=True)

《哈利·波特》系列由J.K.罗琳编著，讲述了年轻的巫师哈利·波特在霍格沃茨魔法学校的成长故事。书中融合了魔法、友谊、勇气与对抗邪恶的主题，深受全球青少年读者的喜爱。此系列不仅在文学上取得了巨大成功，也被改编为商业电影，进一步扩大了其影响力。

In [37]:
# 流式生成回答
for chunk in rag_chain.stream("What is the book abou?"):
    print(chunk, end="", flush=True)

《哈利·波特》系列由J.K.罗琳创作，讲述了一个年轻男孩哈利·波特在霍格沃茨魔法学校的冒险故事。在这里，他不仅发现了自己的魔法能力，还结交了朋友，面对各种挑战和敌人。整个系列探讨了勇气、友情和爱的主题，深受全球读者喜爱。

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [74]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

data_format: {data_format}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [75]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
data_format = """
{
    "book_name": "",
    "author": "",
    "describe": ""
}
"""
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question", "data_format": data_format}).text)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

data_format: 
{
    "book_name": "",
    "author": "",
    "describe": ""
}


Helpful Answer:


In [82]:
data_format = """
{
    "book_name": "",
    "author": "",
    "describe": ""
}
"""

# 如果需要使用 data_format，建议通过某种方式包装成支持的可调用对象
data_format_runnable = RunnablePassthrough(lambda x: data_format)

# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "data_format": data_format_runnable, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [83]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("What is the book about?")

'The book series "Harry Potter," authored by J.K. Rowling, follows the journey of a young wizard, Harry Potter, as he discovers his magical heritage and battles the dark wizard Voldemort. It explores themes of friendship, bravery, and the struggle between good and evil within a richly imagined magical world. Thanks for asking!'

# 在 LangChain Hub 上找一个可用的 RAG 提示词模板

In [43]:
# 使用 hub 模块拉取 rag 提示词模板
# prompt = hub.pull("rlm/rag-prompt")
prompt = hub.pull("homanp/question-answer-pair")

e:\code\github\DjangoPeng\openai-quickstart\.venv\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
# 打印模板
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'data_format', 'number_of_pairs'], input_types={}, partial_variables={}, template='You are an AI assistant tasked with generating question and answer pairs for the given context using the given format. Only answer in the format with no other text. You should create the following number of question/answer pairs: {number_of_pairs}. Return the question/answer pairs as a Python List. Each dict in the list should have the full context provided, a relevant question to the context and an answer to the question.\n\nFormat:\n{data_format}\n\nContext:\n{context}\n'), additional_kwargs={})]


In [88]:
# 为 context 和 question 和 number_of_pairs 填充样例数据，并生成 ChatModel 可用的 Messages
data_format = """
{
    "book_name": "",
    "author": "",
    "describe": ""
}
"""

example_messages = prompt.invoke(
    {"context": "filler context", "data_format": data_format, "number_of_pairs": 1}
).to_messages()

In [90]:
# 查看提示词
print(example_messages[0].content)

You are an AI assistant tasked with generating question and answer pairs for the given context using the given format. Only answer in the format with no other text. You should create the following number of question/answer pairs: 1. Return the question/answer pairs as a Python List. Each dict in the list should have the full context provided, a relevant question to the context and an answer to the question.

Format:

{
    "book_name": "",
    "author": "",
    "describe": ""
}


Context:
filler context



In [94]:
# 如果需要使用 data_format，建议通过某种方式包装成支持的可调用对象
data_format_runnable = RunnablePassthrough(lambda x: data_format)

number_of_pairs_runnable = RunnablePassthrough(lambda x: 2)

# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "data_format": data_format_runnable, "number_of_pairs": number_of_pairs_runnable}
    | prompt
    | llm
    | StrOutputParser()
)

In [95]:
# 流式生成回答
for chunk in rag_chain.stream("What is the book about?"):
    print(chunk, end="", flush=True)

```python
[
    {
        "context": "内容简介\n       · · · · · ·\n\n《哈利·波特(共7册)(精)》编著者J.K.罗琳。     《哈利·波特(共7册)(精)》内容提要：2000年的一个深夜，在美国书店的烛光中，穿着黑斗篷，戴着小眼镜的店员开始销售“哈利o波特4”英文版，从此哈利·波特系列图书席卷全球，也是在这一年，简体中文版“哈利o波特”系列图书通过人民文学出版社引进版权登陆中国，作者J.K.罗琳用一种简单的魔法游戏将魔幻的故事传递给了中国乃至全球的青少年读者，与此同时商业电影的改编更是将这部魔幻题材的小说推向了巅峰。     截止到2013年，哈利·波特系列图书累计销量达到200多万套。     此次，人民文学出版社隆重推出《哈利o波特（精装本）》（全七册），此套精装珍藏版《哈利·波特》听取广大资深哈迷的心声，恢复选用陪伴广大哈迷十年之久深入人心的原有封面，并在此基础上提高工艺、完善装帧，配有独具匠心的精美函套，使...(展开全部)",
        "question": "What is the book about?",
        "answer": "The book series 'Harry Potter' by J.K. Rowling tells the story of a young boy named Harry Potter who discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry. It explores themes of friendship, bravery, love, and the battle against evil."
    }
]
```